In [19]:
import pandas as pd
import numpy as np

In [20]:
c = pd.read_csv('./customer_join.csv')
ul = pd.read_csv('./use_log.csv')

In [21]:
ul.isnull().sum()

log_id         0
customer_id    0
usedate        0
dtype: int64

In [22]:
c.isnull().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
mean                0
median              0
amax                0
amin                0
flag1               0
cal_date            0
mem_period          0
dtype: int64

In [23]:
c.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,amax,amin,flag1,cal_date,mem_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.833333,5.0,8,2,1,2019-04-30,47.968131
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,5.083333,5.0,7,3,1,2019-04-30,47.968131
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.583333,5.0,6,3,1,2019-04-30,47.968131
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,0_종일,10500,2_일반,4.833333,4.5,7,2,1,2019-04-30,47.968131
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,2_야간,6000,2_일반,3.916667,4.0,6,1,1,2019-04-30,47.968131


#### 클러스터 사용 - K-Means

 - 군집 개수 : 4개
 - mean, median, min, max 변수는 월 이용 횟수와 관련된 변수 : 1-8 사이의 값을 가짐
 - mem_period 단위가 너무 크니 Scaling 필요

In [24]:
#사용할 데이터 추출
cc = c[['mean','median','amax','amin','mem_period']]
cc.head()

,mean,median,amax,amin,mem_period
0,4.833333,5.0,8,2,47.968131
1,5.083333,5.0,7,3,47.968131
2,4.583333,5.0,6,3,47.968131
3,4.833333,4.5,7,2,47.968131
4,3.916667,4.0,6,1,47.968131


In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [26]:
#스케일링
scaler = StandardScaler()
x_scaled = scaler.fit_transform(cc)
x_scaled = pd.DataFrame(x_scaled, columns=['mean','median','max','min','mem_period'])
x_scaled.head()

,mean,median,max,min,mem_period
0,-0.281206,-0.133676,0.081177,-0.533620,2.031859
1,-0.140545,-0.133676,-0.379928,-0.021149,2.031859
2,-0.421867,-0.133676,-0.841034,-0.021149,2.031859
3,-0.281206,-0.400393,-0.379928,-0.533620,2.031859
4,-0.796964,-0.667109,-0.841034,-1.046090,2.031859


In [27]:
#K-Means 학습
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(x_scaled)

In [28]:
cc['cluster'] = clusters.labels_
cc.head()

C:\Users\user\AppData\Local\Temp\ipykernel_4444\691509906.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc['cluster'] = clusters.labels_


,mean,median,amax,amin,mem_period,cluster
0,4.833333,5.0,8,2,47.968131,1
1,5.083333,5.0,7,3,47.968131,1
2,4.583333,5.0,6,3,47.968131,1
3,4.833333,4.5,7,2,47.968131,1
4,3.916667,4.0,6,1,47.968131,1


In [32]:
cc.rename(columns={'mean':'월 평균','median':'월 중간 값','amax':'최대 이용 횟수','amin':'최소 이용 횟수','mem_period':'회원 기간'}, inplace= True)

C:\Users\user\AppData\Local\Temp\ipykernel_4444\229478950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cc.rename(columns={'mean':'월 평균','median':'월 중간 값','amax':'최대 이용 횟수','amin':'최소 이용 횟수','mem_period':'회원 기간'}, inplace= True)


In [33]:
#cluster 기준으로 데이터 갯수 세기
cc.groupby('cluster').count()

,월 평균,월 중간 값,최대 이용 횟수,최소 이용 횟수,회원 기간
cluster,,,,,
0,846,846,846,846,846
1,1251,1251,1251,1251,1251
2,759,759,759,759,759
3,1336,1336,1336,1336,1336


그룹 3번에 해당하는 수가 1336명으로 가장 많으며, 다음이 그룹1,0,2 순으로 사람이 많음

In [34]:
# 군집별 특징 확인
cc.groupby('cluster').mean()

,월 평균,월 중간 값,최대 이용 횟수,최소 이용 횟수,회원 기간
cluster,,,,,
0,8.054608,8.039598,10.009456,6.160757,7.809637
1,4.676091,4.670264,7.231815,2.153477,37.819727
2,3.049270,2.882740,4.745718,1.652174,10.147960
3,5.522518,5.373129,8.743263,2.686377,15.688428


- 그룹 0: 평균기간이 7일로 가장 짧고 , 월 평균 횟수 8회로 제일 높음
- 그룹 1,3: 평균 회원 기간이 다른 그룹보다 긺, 그룹1은 그룹3보다 회원유지기간은 길지만, 월 평균 이용횟수는 적음
- 그룹2 : 평균회원유지 기간이 다른 그룹보다 짧음, 평균 이용횟수도 3회로 가장 낮음

->분석을 통해 그룹별 특징을 정의해서 프로모션할 때 사용 가능... 할.. 수도?

### 차원 축소
- feature 가 5개니 2개로(2차원)줄여보자